# Error Analysis

Alex McDonald

In this notebook, we will load the models and examine the precise examples that the models failed on. We will focus more on the false negative examples, as the model falsely declaring a malicious email to be safe is more of a concern.

First, let's load the relevant code from transformer_train.ipynb:

In [11]:
import torch, torch.nn as nn
import numpy as np
import pandas as pd
import nltk
import math
import matplotlib.pyplot as plt

#Load the vocabulary space, adapted from Homework 2
glove_file = "./datasets/glove.6B.100d.txt" #or 50d

embeddings_dict = {}

with open(glove_file, 'r', encoding='utf8') as f:
    for i, line in enumerate(f):
        if i == 0:
            print(line)
        line = line.strip().split(' ')
        word = line[0]
        embed = np.asarray(line[1:], "float")

        embeddings_dict[word] = embed

print('Loaded {} words from glove'.format(len(embeddings_dict)))

embedding_matrix = np.zeros((len(embeddings_dict)+2, 100)) #add 1 for padding

word2id = {}
id2word = {}
for i, word in enumerate(embeddings_dict.keys()):

    word2id[word] = i                                #Map each word to an index
    id2word[i] = word
    embedding_matrix[i] = embeddings_dict[word]      #That index holds the Glove embedding in the embedding matrix

# Our joint vocabulary for both models / sanity check to see if we've loaded it correctly:
print(word2id['the'])
print(embedding_matrix[word2id['the']])

word2id['<pad>'] = embedding_matrix.shape[0] - 2
word2id['<start>'] = embedding_matrix.shape[0] - 1
id2word[embedding_matrix.shape[0] - 2] = '<pad>'
id2word[embedding_matrix.shape[0] - 1] = '<start>'
print(embedding_matrix[word2id['<pad>']])

spam_full = pd.read_csv("./datasets/SpamHam/train.csv")#, nrows=20000)

split_ratio = 0.8
train_size = int(split_ratio*spam_full.shape[0])
spam_train = spam_full.iloc[:train_size]
spam_valid = spam_full.iloc[train_size:]
max_length = 120 #inclusive of start token
'''
train_size_url = int(split_ratio*urls_full.shape[0])
urls_train = urls_full.iloc[:train_size]
urls_valid = urls_full.iloc[train_size:]
url_max_length = 120
'''
start_id = word2id['<start>']

def tokenize_example(line):
    example = [start_id]
    tokenized = nltk.word_tokenize(line)
    i = 0
    for token in tokenized:
        if not (token in word2id): continue #not using <unk> for spam dataset
        i += 1
        if (i >= max_length): break
        example.append(word2id[token])
        
    #add padding
    padding = word2id["<pad>"]
    for i in range(max_length - len(example)):
        example.append(padding)
    return np.array(example)

def tokenize(df):
    examples = []
    for index, row in df.iterrows():
        example = tokenize_example(row["text"])
        if (len(example) > 0 and len(example.shape) > 0): examples.append((example, row["label"]))
    return examples

tokenized_spam_valid = tokenize(spam_valid)

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062

Loaded 400000 words from glove
0
[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0

In [6]:
class TransformerModel(nn.Module):

    def __init__(self, embedding_matrix, model_size, n_heads, n_layers, hidden_size, embedding_dims=100, vocab_size=None):
        super().__init__()

        if not (embedding_matrix is None): #glove
            self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix))
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dims)
        self.pos_encoding = PositionalEncoding(embedding_dims, max_length)
        self.input_linear = nn.Linear(embedding_dims, model_size)
        encoder_layers = nn.TransformerEncoderLayer(model_size, n_heads, hidden_size, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        #self.encoder = nn.Transformer(encoder_layers, n_layers, batch_first=True)
        self.output_hidden_1 = nn.Linear(model_size, hidden_size)
        self.relu = nn.ReLU()
        self.output_hidden_2 = nn.Linear(hidden_size, 2) #binary classification
        self.model_size = model_size

        #initialize
        initrange = 0.1
        self.input_linear.weight.data.uniform_(-initrange, initrange)
        self.input_linear.bias.data.zero_()
        self.output_hidden_1.weight.data.uniform_(-initrange, initrange)
        self.output_hidden_1.bias.data.zero_()
        self.output_hidden_2.weight.data.uniform_(-initrange, initrange)
        self.output_hidden_2.bias.data.zero_()

    def forward(self, input):

        #print("intput.shape: ", input.shape, len(input.shape))
        input = (self.embedding(input) * math.sqrt(self.model_size)) #recommended from documentation
        input = self.pos_encoding(input)
        #print("input after poe:", input.shape)

        input = self.input_linear(input) #get a representation that has the model size for the positionally encoded embeddings
        #print("after input linear: ", input.shape)
        
        output = self.encoder(input)[:,0] #take the last vector
        #print("after encoder: ", output.shape)
        output = self.output_hidden_1(output)

        output = self.relu(output)
        output = self.output_hidden_2(output)

        #print("after linear:", output.shape)

        return output

class PositionalEncoding(nn.Module):

    def __init__(self, model_size, max_len): #from torch documentation
        super().__init__()
        self.dropout = nn.Dropout(p=0.1)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, model_size, 2) * (-math.log(10000.0) / model_size))
        pe = torch.zeros(max_len, 1, model_size)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

def predict(model, valid_dataloader): #modified to save examples the model fails on

    softmax = nn.Softmax()

    model.eval()
    
    sigmoid = nn.Sigmoid()

    total_examples = 0
    false_negative = []
    false_positive = []

    for x, y in valid_dataloader:
        x = x.squeeze()
        if (len(x.shape) == 0): continue
        output = model(x)
        output = softmax(output)

        for i in range(output.shape[0]):
            if (output[i][0].item() >= 0.5):
                if (y[i].item() != 0):
                    false_negative.append(untokenize(x[i]))
            elif (y[i].item() == 0):
                false_positive.append(untokenize(x[i]))
        total_examples += output.shape[0]
        #print("total examples:", total_examples, "; T+:", true_positive, "; F+:", false_positive, "; T-:", true_negative, "; F-:", false_negative)

    return [false_negative, false_positive]

We will also create a function that takes in word IDs and translates them back to tokens.

In [15]:
def untokenize(example):
    s = ""
    for i in range(len(example)):
        s += id2word[example[i].item()] + " "
    return s

## Spam Model

In [17]:
epochs = 6
batch_size = 32
print_frequency = 250
n_heads = 2
n_layers = 2
model_size = 28
hidden_size = 48
pos_weight_coeff = 1.05
lr = 0.001

spam_model = TransformerModel(embedding_matrix, model_size=model_size, n_heads=n_heads, n_layers=n_layers, hidden_size=hidden_size)
spam_model.load_state_dict(torch.load('./trained_models/spam.pt'))

spam_valid_dataloader = torch.utils.data.DataLoader(tokenized_spam_valid, batch_size=batch_size)
false_negative, false_positive = predict(spam_model, spam_valid_dataloader)


In [26]:
print("Randomly selected FALSE NEGATIVE examples: (%s)" % len(false_negative))
for i in np.random.choice(np.arange(0, len(false_negative)), 6):
    print("Example %s: %s" % (i, false_negative[i]) + "\n\n")

Randomly selected FALSE NEGATIVE examples: (427)
Example 33: <start> two dynamic mlas harvey barry <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 


Example 332: <start> take that <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [27]:
print("Randomly selected FALSE POSITIVE examples: (%s)" % len(false_positive))
for i in np.random.choice(np.arange(0, len(false_positive)), 6):
    print("Example %s: %s" % (i, false_positive[i]) + "\n\n")

Randomly selected FALSE POSITIVE examples: (608)
Example 13: <start> test <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 


Example 349: <start> is n't he just a cutie pie <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>